In [51]:
import cv2
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
import matplotlib.pyplot as plt
import time

In [56]:
# Path to dataset: structured as "dataset/A/A1.jpg", "dataset/B/B2.jpg", etc.
DATASET_PATH = "C:/Users/megan/Downloads/asl_dataset"  #Replace with your dataset path
IMAGE_SIZE = (100, 100)

def load_images_and_labels(dataset_path):
    X = []
    y = []
    for label in sorted(os.listdir(dataset_path)):
        label_path = os.path.join(dataset_path, label)
        if not os.path.isdir(label_path):
            continue
        for img_name in os.listdir(label_path):
            img_path = os.path.join(label_path, img_name)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is None:
                continue
            img = cv2.resize(img, IMAGE_SIZE)
            X.append(img)
            y.append(label)
    return np.array(X), np.array(y)

# Load data
print("Loading images...")
X, y = load_images_and_labels(DATASET_PATH)

Loading images...


In [57]:
# Example: reshape and normalize data (assuming 28x28 grayscale images)
# Update these shapes according to your actual data
X = X.reshape(-1, 100, 100, 1).astype("float32")/255.0  # Normalize and reshape

from collections import Counter

print("Label distribution:", Counter(y))

# Encode labels 
le = LabelEncoder()
y_encoded = le.fit_transform(y)
y_categorical = to_categorical(y_encoded)

print("y_encoded shape:", y_encoded.shape)
print("y_categorical shape:", y_categorical.shape)

print("X shape:", X.shape)
print("First label:", y[0])
print("Encoded labels:", y_encoded[:5])

# Split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.2, shuffle=True, stratify=y_encoded)

# Define CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(y_categorical.shape[1], activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train CNN with timing
print("Training classifier...")
start_train = time.time()
model.fit(X_train, y_train, epochs=30, batch_size=32, verbose=1)
end_train = time.time()
print(f"Training completed in {end_train - start_train:.4f} seconds.")

# Evaluate CNN with timing
print("Evaluating...")
start_eval = time.time()
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(y_test, axis=1)
end_eval = time.time()
print(f"Evaluation completed in {end_eval - start_eval:.4f} seconds.")
print(classification_report(y_true, y_pred))


Label distribution: Counter({'a': 70, 'b': 70, 'c': 70, 'd': 70, 'e': 70, 'f': 70, 'g': 70, 'h': 70, 'i': 70, 'j': 70, 'k': 70, 'l': 70, 'm': 70, 'n': 70, 'o': 70, 'p': 70, 'q': 70, 'r': 70, 's': 70, 'u': 70, 'v': 70, 'w': 70, 'x': 70, 'y': 70, 'z': 70, 't': 65})
y_encoded shape: (1815,)
y_categorical shape: (1815, 26)
X shape: (1815, 100, 100, 1)
First label: a
Encoded labels: [0 0 0 0 0]
Training classifier...


C:\Users\megan\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
46/46 ━━━━━━━━━━━━━━━━━━━━ 14s 212ms/step - accuracy: 0.2465 - loss: 2.5824
Epoch 2/30
46/46 ━━━━━━━━━━━━━━━━━━━━ 10s 214ms/step - accuracy: 0.8168 - loss: 0.5547
Epoch 3/30
46/46 ━━━━━━━━━━━━━━━━━━━━ 10s 217ms/step - accuracy: 0.9073 - loss: 0.2531
Epoch 4/30
46/46 ━━━━━━━━━━━━━━━━━━━━ 10s 215ms/step - accuracy: 0.9575 - loss: 0.1558
Epoch 5/30
46/46 ━━━━━━━━━━━━━━━━━━━━ 10s 215ms/step - accuracy: 0.9509 - loss: 0.1525
Epoch 6/30
46/46 ━━━━━━━━━━━━━━━━━━━━ 10s 215ms/step - accuracy: 0.9676 - loss: 0.1065
Epoch 7/30
46/46 ━━━━━━━━━━━━━━━━━━━━ 10s 215ms/step - accuracy: 0.9804 - loss: 0.0498
Epoch 8/30
46/46 ━━━━━━━━━━━━━━━━━━━━ 10s 214ms/step - accuracy: 0.9881 - loss: 0.0471
Epoch 9/30
46/46 ━━━━━━━━━━━━━━━━━━━━ 10s 215ms/step - accuracy: 0.9775 - loss: 0.0697
Epoch 10/30
46/46 ━━━━━━━━━━━━━━━━━━━━ 10s 215ms/step - accuracy: 0.9793 - loss: 0.0572
Epoch 11/30
46/46 ━━━━━━━━━━━━━━━━━━━━ 10s 214ms/step - accuracy: 0.9856 - loss: 0.0323
Epoch 12/30
46/46 ━━━━━━━━━━━━━━━━━━━━ 10

In [58]:
test_loss, test_accuracy = model.evaluate(X_test,y_test, verbose = 1)
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Loss: {test_loss:.4f}")

12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 85ms/step - accuracy: 0.9872 - loss: 0.0455
Test Accuracy: 0.9862
Test Loss: 0.0459
